In [76]:
G = \
[
    [None, 2, 1, None],
    [2, None, 1, 1],
    [1, 1, None, 4],
    [None, 1, 4, None]
]

In [77]:
class BFS:
    def __init__(self, G):
        self.was_in = []
        self.G = G
        
    def visit(self, v):
        self.was_in += [v]
        for i in range(len(self.G[v])):
            if self.G[v][i] is not None and i not in self.was_in:
                self.queue += [i]
                yield(v, i)
                
    def start(self, v):
        self.queue = [v]
        while len(self.queue) > 0:
            v = self.queue[0]
            self.queue = self.queue[1:]
            if v not in self.was_in:
                yield from self.visit(v)
            

In [78]:
for f,t in BFS(G).start(0):
    print(f, t)

0 1
0 2
1 2
1 3
2 3


In [41]:
class FindPaths(BFS):
    def __init__(self, G):
        super().__init__(G)
        self.edges = [[] for _ in range(len(self.G))]
        
    def start(self, v, e):
        self.edges[v] =[[]]
        for f, t in super().start(v):
            for edge in self.edges[f]:
                if t not in edge:
                    self.edges[t] += [edge + [f]]
            for edge in self.edges[t]:
                if f not in edge:
                    self.edges[f] += [edge + [t]]
        return [edge + [e] for edge in self.edges[e]]

In [42]:
for edge in FindPaths(G).start(0, 3):
    print(edge)

[0, 1, 3]
[0, 2, 1, 3]
[0, 2, 3]
[0, 1, 2, 3]


In [53]:
def zeroes_like(G):
    return [[0 for _ in range(len(G[0]))] for _ in range(len(G))]

def vertice_pairs(edge):
    return zip(edge, edge[1:])

class FordFulkenson(FindPaths):
    def __init__(self, G):
        super().__init__(G)
        self.flows = zeroes_like(G)
        
    def path_increase(self, edge):
        return min([self.G[e][n] - self.flows[e][n] for e, n in vertice_pairs(edge)])
        
    def start(self, v, e):
        for edge in super().start(v, e):
            increase = self.path_increase(edge)
            print(edge, increase)
            if increase > 0:
                for f, t in vertice_pairs(edge):
                        self.flows[f][t] += increase
        print(self.flows)
        return sum([row[e] for row in self.flows])

In [54]:
FordFulkenson(G).start(0, 3)

[0, 1, 3] 1
[0, 2, 1, 3] 0
[0, 2, 3] 1
[0, 1, 2, 3] 1
[[0, 2, 1, 0], [0, 0, 1, 1], [0, 0, 0, 2], [0, 0, 0, 0]]


3

In [109]:
def deep_dup(G):
    return [[i or 0 for i in row] for row in G]

class RealFordFulkenson:
    def __init__(self, G):
        self.G = G
        self.flows = zeroes_like(G)
    
    def potential_increase(self, e, n):
        return self.G[e][n] - self.flows[e][n]
        
    def path_increase(self, edge):
        return min([self.potential_increase(e, n) for e, n in vertice_pairs(edge)])
    
    def recover_path(self, f):
        p = []
        while f is not None:
            p = [f] + p
            f = self.previous[f]
        return p
    
    def bfs(self, v):
        self.was_in = []
        self.previous = [None for _ in range(len(self.G[0]))]
        self.queue = [v]
        while len(self.queue) > 0:
            v = self.queue[0]
            self.queue = self.queue[1:]
            if v not in self.was_in:
                    self.visit(v)
                
    def visit(self, v):
        self.was_in += [v]
        for i in range(len(self.G[v])):
            if self.G[v][i] is not None and i not in self.was_in and self.potential_increase(v, i) > 0 and self.previous[i] is None:
                self.queue += [i]
                self.previous[i] = v
    
    def start(self, v, e):
        max_flow = 0
        while True:
            self.bfs(v)
            if self.previous[e] is None:
                break
            
            edge = self.recover_path(e)
            increase = self.path_increase(edge)
            if increase == 0:
                continue
            print(edge, increase)
            max_flow += increase
            for x, y in vertice_pairs(edge):
                self.flows[x][y] += increase
                self.flows[y][x] -= increase

                
        print(self.flows)
        return max_flow

In [111]:
f, t = [0, 3]
RealFordFulkenson(G).start(f, t), FordFulkenson(G).start(f, t)

[0, 1, 3] 1
[0, 2, 3] 1
[0, 1, 2, 3] 1
[[0, 2, 1, 0], [-2, 0, 1, 1], [-1, -1, 0, 2], [0, -1, -2, 0]]
[0, 1, 3] 1
[0, 2, 1, 3] 0
[0, 2, 3] 1
[0, 1, 2, 3] 1
[[0, 2, 1, 0], [0, 0, 1, 1], [0, 0, 0, 2], [0, 0, 0, 0]]


(3, 3)